In [1]:
#데이터를 가져오는 부분

from keras.datasets import boston_housing
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()
#keras에서는 학습 데이터와 라벨 / 검증 데이터와 라벨을 제공해준다.

57026/57026 [==============================] - 0s 0us/step


In [9]:
print(train_data.shape)
print(test_data.shape)
#데이터는 404개가 있고 차원은 13개=데이터셋의 열의 수가=13 데이터 특징이 13개

(404, 13)
(102, 13)


In [4]:
print(train_labels.shape)
print(test_labels.shape)
print(train_labels[0])

(404,)
(102,)
15.2


In [5]:
print(train_data[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [6]:
#데이터 전처리 (Scaling)
#데이터 항목에서 단순이 수치 기준이 달라서 편향값이 생길 수 있으므로 스케일링을 해준다.

#데이터 전처리
mean = train_data.mean(axis=0)
#asix=0은 각 컬럼의 평균
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [10]:
#모델의 구성

from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  #레이어들이 일렬로 연결되도록 하는 함수
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
  #train_data.shape[1]은 데이터 모델은 튜플로 나오는데 [1]을 통해서 열의 개수를 알려준다.
  #(404, 13)에서 13을 의미
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',
                loss='mse',
                metrics=['mse'])
  
  return model


In [12]:
#k-folder 검증을 사용한 훈련 검증
#데이터 모집이 적을 때 쓰는 기법
#데이터를 등분한 후에 번갈아가면서 학습과 검증을 반복

import numpy as np

k = 4 #folder 개수
num_val_samples = len(train_data) // 4   #한개의 folder 데이터 수(101)
all_scores = []

for i in range(k):
  print("처리중인 폴드 #", i, "\n")

  #검증 데이터의 준비
  #시작점(1이면 101, 2이면 202, 3이면 303 ,4이면 404)
  #끝점은 다음 배열의 시작점
  val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]  
  val_labels = train_labels[i * num_val_samples : (i+1) * num_val_samples]    
  
  print("검증 데이터/라벨 : ", i * num_val_samples, (i+1) * num_val_samples) 


  #학습 데이터의 준비
  data1 = train_data[:i * num_val_samples]
  label1 = train_labels[:i * num_val_samples]
  data2 = train_data[(i+1) * num_val_samples:]
  label2 = train_labels[(i+1) * num_val_samples:]

  partial_train_data = np.concatenate([data1, data2], axis=0)
  partial_train_labels = np.concatenate([label1, label2], axis=0)
  
  print("학습 데이터/라벨 : ", 0, i * num_val_samples)
  print("학습 데이터/라벨 : ", (i+1) * num_val_samples, 404, "\n")

  # 모델의 학습
  model = build_model()
  model.summary()

  model.fit(partial_train_data, partial_train_labels, epochs=500, batch_size=128, verbose=0)

  # 모델의 검증
  val_mse, val_mae = model.evaluate(val_data, val_labels)
  print(val_mse, val_mae)
  all_scores.append(val_mae)

처리중인 폴드 # 0 

검증 데이터/라벨 :  0 101
학습 데이터/라벨 :  0 0
학습 데이터/라벨 :  101 404 

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                896       
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 5ms/step - loss: 8.1041 - mse: 8.1041
8.10413646697998 8.10413646697998
처리중인 폴드 # 1 

검증 데이터/라벨 :  101 202
학습 데이터/라벨 :  0 101
학습 데이터/라벨 :  202 404 

Model: "sequential_5"
______________________________

In [13]:
#각 검증/학습을 거친 mse평가 수치
all_scores

[8.10413646697998, 13.077174186706543, 11.970891952514648, 9.930956840515137]

In [14]:
np.mean(all_scores)

10.770789861679077